## Sesión 4 

### Grid Search - Sklearn

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as plt
import seaborn as sns
import time
import pyarrow

from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline 

In [2]:
## Ecobici
#start = time.time()
#ecobici = pd.read_parquet("ecobici/ecobici_2010_2017_parquet/")
#print("tiempo lectura parquet: ", (time.time() - start)/60)
#ecobici.shape

In [3]:
#ecobici.edad_usuario = ecobici.edad_usuario.astype('int')


## Sklearn

Solo para ver las semejanzas y diferencias entre el GridParam de Sklearn, SparkML y H2Oml.

Cargaremos el dataset de cáncer de mama que viene incluido en sklearn, se trata de un problema supervisado de clasificación con 2 posibles categorías B (benigno) o M (maligno), 569 observaciones, 30 variables.

In [4]:
from sklearn import datasets

data, target = datasets.load_breast_cancer(return_X_y=True)
(data.shape, target.shape)

((569, 30), (569,))

In [5]:
data[0:5,]

array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
        8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
        3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
        1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
        1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e

In [6]:
target[0:5,]

array([0, 0, 0, 0, 0])

In [7]:
models = {'DT': DecisionTreeClassifier(), 
          'RF': RandomForestClassifier(n_jobs=-1),
          'LR': LogisticRegression(n_jobs=-1)}

grid = {'DT': {'min_samples_leaf': [5,7,9]},
        'RF': {'n_estimators': [300,500,800],
                 'min_samples_leaf': [5,7,9]},
        'LR': {'penalty': ['l1','l2']}}

In [8]:
models_to_run = ['LR','DT','RF']

In [9]:
#tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                     'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
#tuned_parameters

In [64]:
start_magic_loop = time.time()
for model_to_try in models_to_run:
    start_model = time.time()
    parameter_values = grid[model_to_try]
    for parameter_grid in parameter_values:
        start_hp = time.time()
        m = GridSearchCV(estimator=models[model_to_try], 
                         param_grid=parameter_grid, 
                         cv=2, 
                         scoring="f1")
        m.fit(X_train, y_train)
        print("hp tunning: ", (time.time() - start_hp)/60)
    print("modelo: ", model_to_try, " ", (time.time() - start_model)/60)
print("magic loop: ", (time.time() - start_magic_loop)/60)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False) {'penalty': ['l1', 'l2']}


/home/lmillan/.pyenv/versions/3.6.0/envs/h20/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lmillan/.pyenv/versions/3.6.0/envs/h20/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/home/lmillan/.pyenv/versions/3.6.0/envs/h20/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lmillan/.pyenv/versions/3.6.0/envs/h20/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_

hp tunning:  0.017726425329844156
modelo:  LR   0.017736566066741944
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best') {'min_samples_leaf': [5, 7, 9]}
hp tunning:  0.001417843500773112
modelo:  DT   0.0014215469360351562
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False) {'n_estimators': [300, 500, 800], 'min_samples_leaf':

### Grid Search SparkML

In [7]:
import findspark
findspark.init()


from pyspark.sql import functions
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

spark = SparkSession.builder \
.master("local") \
.appName("example") \
.getOrCreate()

In [8]:
# Prepare training documents, which are labeled.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

# Prepare test documents, which are unlabeled.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    print(row)

Row(id=4, text='spark i j k', probability=DenseVector([0.2581, 0.7419]), prediction=1.0)
Row(id=5, text='l m n', probability=DenseVector([0.9186, 0.0814]), prediction=0.0)
Row(id=6, text='mapreduce spark', probability=DenseVector([0.432, 0.568]), prediction=1.0)
Row(id=7, text='apache hadoop', probability=DenseVector([0.6766, 0.3234]), prediction=0.0)


+ $\rightarrow$ Intenta hacer tu magic loop con una regresión logística, un árbol y un RF.
+ ¿Cuál es el "dibujo" asociado a este flujo de trabajo con estimators y transformers?